In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/practice-hackathon/sample_submission.csv
/kaggle/input/practice-hackathon/Train.csv
/kaggle/input/practice-hackathon/Test.csv


In [ ]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [18]:
# Load the training and test data
train_data = pd.read_csv('/kaggle/input/practice-hackathon/Train.csv')
test_data = pd.read_csv('/kaggle/input/practice-hackathon/Test.csv')

In [19]:
# Split the training data into features and target variable
X_train = train_data.drop('Target', axis=1)
y_train = train_data['Target']

In [20]:
# Identify categorical and numerical columns
categorical_cols = ['A']
numerical_cols = ['B', 'C', 'D', 'E', 'F', 'G']

In [22]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [23]:
preprocessor = ColumnTransformer(
    transformers=[
        ('imputer', SimpleImputer(strategy='median'), numerical_cols),
        ('scaler', StandardScaler(), numerical_cols),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])


In [25]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('adaboost', AdaBoostClassifier(
        n_estimators=100,
        random_state=42
    ))
])


In [26]:
pipeline.fit(X_train, y_train_encoded)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('imputer',
                                                  SimpleImputer(strategy='median'),
                                                  ['B', 'C', 'D', 'E', 'F',
                                                   'G']),
                                                 ('scaler', StandardScaler(),
                                                  ['B', 'C', 'D', 'E', 'F',
                                                   'G']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['A'])])),
                ('adaboost',
                 AdaBoostClassifier(n_estimators=100, random_state=42))])

In [27]:
predictions_encoded = pipeline.predict(test_data)
predictions = label_encoder.inverse_transform(predictions_encoded)

In [29]:
submission = pd.DataFrame({
    'Index': test_data['Index'],
    'Target': predictions
})
submission

,Index,Target
0,866,9
1,1483,9
2,599,9
3,1702,9
4,670,7
...,...,...
1040,532,7
1041,3417,9
1042,1505,9
1043,2245,7


In [30]:
submission.to_csv('submission.csv', columns=['Index', 'Target'], index=False)